Referências:

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

https://github.com/s-miller/Spam-classification-comparing-Naive-Bayes-vs-random-forest-and-playing-around-with-sklearn-pipeline/blob/master/Spam%20classification%20comparing%20Naive%20Bayes%20vs%20random%20forest%20(and%20playing%20around%20with%20sklearn%20pipelining).ipynb

In [28]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [29]:
def process_text(txt:str):
    '''Função que recebe o tweet e remove stopwords'''
    return [word for word in txt.split() if word not in stopwords.words('portuguese')]

In [30]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

In [31]:
tt = pd.read_csv('DebateGovRS_Anotado.xlsx - DebateGovRS_Anotado.csv')
tt.head()

,author_id,created_at,id,lang,retweet_count,text,sinal
0,"1,46E+18",2022-08-08T01:08:23Z,"1,56E+18",pt,45,RT @EdegarPretto O meu candidato vice-governad...,1.0
1,"1,51E+18",2022-08-08T01:08:23Z,"1,56E+18",pt,0,Periga o Argenta chamar o Simon pra responder ...,NaN
2,"1,06E+18",2022-08-08T01:08:21Z,"1,56E+18",pt,14,RT @WinstonLing Segue o link para assistirem o...,1.0
3,"7,15E+17",2022-08-08T01:08:20Z,"1,56E+18",pt,88,RT @EdegarPretto Eu quero dizer aos gaúchos e ...,NaN
4,1455004730,2022-08-08T01:08:20Z,"1,56E+18",pt,0,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0.0


In [32]:
'''Seleciona somente os tweets classificados'''
tt = tt.loc[tt['sinal'].notna()]
tt.head()

,author_id,created_at,id,lang,retweet_count,text,sinal
0,"1,46E+18",2022-08-08T01:08:23Z,"1,56E+18",pt,45,RT @EdegarPretto O meu candidato vice-governad...,1.0
2,"1,06E+18",2022-08-08T01:08:21Z,"1,56E+18",pt,14,RT @WinstonLing Segue o link para assistirem o...,1.0
4,1455004730,2022-08-08T01:08:20Z,"1,56E+18",pt,0,@wmfigueiredo @evertonvf @bandrs @EduardoLeite...,0.0
5,"1,23E+18",2022-08-08T01:08:20Z,"1,56E+18",und,0,@EduardoLeite_ 🤡,0.0
6,116829646,2022-08-08T01:08:19Z,"1,56E+18",pt,41,Nós já governamos o Rio Grande Sul com @olivio...,0.0


In [33]:
'''Remove acentos e caracteres especiais / Coloca em minúscula'''
tt['text'] = tt['text'].str.lower().str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')
tt['text']

0       rt @edegarpretto o meu candidato vice-governad...
2       rt @winstonling segue o link para assistirem o...
4       @wmfigueiredo @evertonvf @bandrs @eduardoleite...
5                                        @eduardoleite_  
6       nos ja governamos o rio grande sul com @olivio...
                              ...                        
2918               @onyxlorenzoni carai chevrolet ta puto
2919    agora e pau cu dos liberaloides fogo nos fasci...
2922    @onyxlorenzoni ele e bom historiador  pena que...
2924    #debatenabandrs la vem mesmas lenga-lengas de ...
2949                       @ricardojobim30 sucesso debate
Name: text, Length: 778, dtype: object

In [34]:
'''Remove '\n', @, RT, #'''
tt['text'] = tt['text'].replace(to_replace=r'\\n',value=' ',regex=True)
tt['text'] = tt['text'].replace(to_replace=r'(?<!\w)@[\w+]{1,100}\b',value='',regex=True)
tt['text'] = tt['text'].replace(to_replace=r'^rt\s',value='',regex=True)
tt['text'] = tt['text'].replace(to_replace=r'(?<!\w)#[\w+]{1,100}\b',value='',regex=True)
tt

,author_id,created_at,id,lang,retweet_count,text,sinal
0,"1,46E+18",2022-08-08T01:08:23Z,"1,56E+18",pt,45,o meu candidato vice-governador e o fundador...,1.0
2,"1,06E+18",2022-08-08T01:08:21Z,"1,56E+18",pt,14,segue o link para assistirem o debate de hoje...,1.0
4,1455004730,2022-08-08T01:08:20Z,"1,56E+18",pt,0,aos que gostam governador atual quando for...,0.0
5,"1,23E+18",2022-08-08T01:08:20Z,"1,56E+18",und,0,,0.0
6,116829646,2022-08-08T01:08:19Z,"1,56E+18",pt,41,nos ja governamos o rio grande sul com e e n...,0.0
...,...,...,...,...,...,...,...
2918,"1,27E+18",2022-08-08T00:02:36Z,"1,56E+18",pt,0,carai chevrolet ta puto,0.0
2919,"1,51E+18",2022-08-08T00:02:35Z,"1,56E+18",pt,0,agora e pau cu dos liberaloides fogo nos fasci...,0.0
2922,"1,52E+18",2022-08-08T00:02:26Z,"1,56E+18",pt,0,ele e bom historiador pena que militonto,0.0
2924,"1,51E+18",2022-08-08T00:02:22Z,"1,56E+18",pt,0,la vem mesmas lenga-lengas de sempre priorida...,0.0


In [35]:
'''Remove espaços desnecessários'''
tt['text'] = tt['text'].str.rstrip().str.lstrip().str.replace('  ',' ')

In [36]:
'''Remove textos vazios'''
tt = tt[tt['text']!='']
tt

,author_id,created_at,id,lang,retweet_count,text,sinal
0,"1,46E+18",2022-08-08T01:08:23Z,"1,56E+18",pt,45,o meu candidato vice-governador e o fundador t...,1.0
2,"1,06E+18",2022-08-08T01:08:21Z,"1,56E+18",pt,14,segue o link para assistirem o debate de hoje ...,1.0
4,1455004730,2022-08-08T01:08:20Z,"1,56E+18",pt,0,aos que gostam governador atual quando forem p...,0.0
6,116829646,2022-08-08T01:08:19Z,"1,56E+18",pt,41,nos ja governamos o rio grande sul com e e nos...,0.0
7,337744562,2022-08-08T01:08:18Z,"1,56E+18",pt,1,onyx caixa 2 confesso nao pode falar em corrup...,0.0
...,...,...,...,...,...,...,...
2918,"1,27E+18",2022-08-08T00:02:36Z,"1,56E+18",pt,0,carai chevrolet ta puto,0.0
2919,"1,51E+18",2022-08-08T00:02:35Z,"1,56E+18",pt,0,agora e pau cu dos liberaloides fogo nos fasci...,0.0
2922,"1,52E+18",2022-08-08T00:02:26Z,"1,56E+18",pt,0,ele e bom historiador pena que militonto,0.0
2924,"1,51E+18",2022-08-08T00:02:22Z,"1,56E+18",pt,0,la vem mesmas lenga-lengas de sempre prioridad...,0.0


In [37]:
'''Remove registros repetidos que devem ser removidos'''
tt = tt.drop_duplicates(subset=['text'])
tt.reset_index(drop=True,inplace=True)
tt

,author_id,created_at,id,lang,retweet_count,text,sinal
0,"1,46E+18",2022-08-08T01:08:23Z,"1,56E+18",pt,45,o meu candidato vice-governador e o fundador t...,1.0
1,"1,06E+18",2022-08-08T01:08:21Z,"1,56E+18",pt,14,segue o link para assistirem o debate de hoje ...,1.0
2,1455004730,2022-08-08T01:08:20Z,"1,56E+18",pt,0,aos que gostam governador atual quando forem p...,0.0
3,116829646,2022-08-08T01:08:19Z,"1,56E+18",pt,41,nos ja governamos o rio grande sul com e e nos...,0.0
4,337744562,2022-08-08T01:08:18Z,"1,56E+18",pt,1,onyx caixa 2 confesso nao pode falar em corrup...,0.0
...,...,...,...,...,...,...,...
594,"1,27E+18",2022-08-08T00:02:36Z,"1,56E+18",pt,0,carai chevrolet ta puto,0.0
595,"1,51E+18",2022-08-08T00:02:35Z,"1,56E+18",pt,0,agora e pau cu dos liberaloides fogo nos fasci...,0.0
596,"1,52E+18",2022-08-08T00:02:26Z,"1,56E+18",pt,0,ele e bom historiador pena que militonto,0.0
597,"1,51E+18",2022-08-08T00:02:22Z,"1,56E+18",pt,0,la vem mesmas lenga-lengas de sempre prioridad...,0.0


In [38]:
'''Cria o bag of words '''
bow_transformer = CountVectorizer(analyzer=process_text)
df_bow = bow_transformer.fit_transform(tt.text)

In [40]:
print(bow_transformer.vocabulary_)

{'candidato': 370, 'vice-governador': 2322, 'fundador': 1048, 'trabalhismo': 2202, 'defensor': 638, 'escola': 855, 'integral': 1184, 'leonel': 1268, 'bri...': 333, 'segue': 2040, 'link': 1285, 'assistirem': 214, 'debate': 627, 'hoje': 1122, 'internet': 1190, 'vamos': 2269, 'apoiar': 180, 'proximo': 1841, 'governador': 1090, 'ba...': 253, 'gostam': 1082, 'atual': 234, 'posto': 1741, 'combustivel': 477, 'paguem': 1582, 'valor': 2264, 'reducao': 1921, 'icms': 1137, 'pois': 1721, 'nao': 1467, 'quer': 1880, 'ja': 1207, 'governamos': 1092, 'rio': 1990, 'grande': 1101, 'sul': 2128, 'nunca': 1517, 'aumentamos': 238, 'impostos': 1154, 'vendemos': 2290, 'patrimonio': 1633, 'gauchos': 1070, 'atrasamos': 228, 'salario': 2025, 'servidores': 2072, ']': 56, 'onyx': 1545, 'caixa': 352, '2': 21, 'confesso': 511, 'pode': 1715, 'falar': 941, 'corrupcao': 563, 'fecho': 970, 'edegarrrrr': 779, 'gente': 1072, 'bem': 285, 'mesma': 1375, 'intolerante': 1191, 'mata': 1341, 'preferencia': 1767, 'politica': 1725

In [41]:
'''tfid'''
tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(df_bow)

TfidfTransformer()

In [42]:
messages_tfidf = tfidf_transformer.transform(df_bow)

In [43]:
x = messages_tfidf
y = tt.sinal

In [45]:
x_train, x_test, y_train, y_test = train_test_split(tt.text,tt.sinal,test_size=0.3)

In [46]:
pipeline = Pipeline([
        ('bow', CountVectorizer(analyzer=process_text)),
        ('tfidf', TfidfTransformer()),
        ('classifier', SVC())
    ])

In [47]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function process_text at 0x000001F0A4EAD0D0>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

In [48]:
y_pred = pipeline.predict(x_test)

In [49]:
print(confusion_matrix(y_test,y_pred))

[[108   5]
 [ 43  24]]


In [50]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.72      0.96      0.82       113
         1.0       0.83      0.36      0.50        67

    accuracy                           0.73       180
   macro avg       0.77      0.66      0.66       180
weighted avg       0.76      0.73      0.70       180



In [51]:
pipeline_rfc = Pipeline([
        ('bow', CountVectorizer(analyzer=process_text)),
        ('tfidf', TfidfTransformer()),
        ('classifier', RandomForestClassifier(50))
    ])

In [52]:
pipeline_rfc.fit(x_train, y_train)
y_pred = pipeline_rfc.predict(x_test)

In [53]:
print(confusion_matrix(y_test,y_pred))

[[103  10]
 [ 37  30]]


In [54]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.74      0.91      0.81       113
         1.0       0.75      0.45      0.56        67

    accuracy                           0.74       180
   macro avg       0.74      0.68      0.69       180
weighted avg       0.74      0.74      0.72       180

